In [31]:
# Installiere benötigte Pakete für Python 3.12
import sys
!"{sys.executable}" -m pip install --upgrade --force-reinstall paramiko sshtunnel openai pillow PyPDF2 pdf2image

  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached sshtunnel-0.4.0-py2.py3-none-any.whl.metadata (19 kB)
  Using cached openai-1.65.2-py3-none-any.whl.metadata (27 kB)
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached cryptography-44.0.2-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached PyNaCl-1.5.0-cp36-abi3-win_amd64.whl.metadata (8.7 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.
pyopenssl 24.1.0 requires cryptography<43,>=41.0.5, but you have cryptography 44.0.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import json
import base64
from io import BytesIO
from PyPDF2 import PdfReader
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from openai import OpenAI
from PIL import Image as PILImage
import IPython.display as display
import re
from pdf2image import convert_from_path

In [8]:
# Konfigurationswerte
PDF_DIR = os.path.join(os.getcwd(), "Paper")
IMG_PATH = os.path.join(os.getcwd(), "IMG")
OUTPUT_JSON = "extracted_formulas.json"

# Pixtral-Konfigurationswerte
SSH_HOST = "141.75.89.10"
SSH_PORT = 22
SSH_USERNAME = "oppelfe89127"
LOCAL_PORT_PIXTRAL = 8092
REMOTE_PORT_PIXTRAL = 8092
PIXTRAL_API_BASE = "http://127.0.0.1:8092"
OPENAI_API_KEY = "EMPTY"
OPENAI_API_BASE = "http://127.0.0.1:8092/v1"

# Sicherstellen, dass der IMG_PATH existiert
os.makedirs(IMG_PATH, exist_ok=True)

In [11]:
# Pixtral-Verbindungseinstellungen
def query_pixtral_with_ssh(image_path):
    openai_api_key = OPENAI_API_KEY
    openai_api_base = OPENAI_API_BASE
    
    print(f"Verbindung zu Pixtral API unter {openai_api_base} mit API Key {openai_api_key[:5]}***")
    
    client = OpenAI(api_key=openai_api_key, base_url=openai_api_base)
    
    with SSHTunnelForwarder(
        (SSH_HOST, SSH_PORT),
        ssh_username=SSH_USERNAME,
        local_bind_address=('localhost', LOCAL_PORT_PIXTRAL),
        remote_bind_address=('localhost', REMOTE_PORT_PIXTRAL)
    ) as tunnel:
        tunnel.start()
        print(f"SSH-Tunnel gestartet: localhost:{LOCAL_PORT_PIXTRAL} -> {REMOTE_PORT_PIXTRAL}")
        model = client.models.list().data[0].id
        print(f"Verwendetes Modell: {model}")

        with PILImage.open(image_path) as image:
            buffered = BytesIO()
            image.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            print(f"Bild {image_path} erfolgreich geladen und kodiert.")

            response = client.chat.completions.create(
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": "Suche und Extrahiere ausschließlich mathematische Formeln aus diesem Bild. Gib die Formeln als Latex Formeln zurück. Falls keine Formeln gefunden werden, gib bitte eine die Meldung 'keine Formeln gefunden' zurück."},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_str}"}},
                    ]},
                ],
                model=model,
                max_tokens=512,
            )
            print("Antwort von Pixtral erhalten.")
            return response.choices[0].message.content

In [3]:
# Funktion zur Umwandlung von PDF-Seiten in Bilder
def extract_images_from_pdf(pdf_path):
    poppler_path = os.path.join(os.getcwd(), "poppler-24.08.0", "Library", "bin")
    print(f"Konvertiere PDF {pdf_path} in Bilder...")
    images = convert_from_path(pdf_path, poppler_path=poppler_path)
    image_paths = []
    for i, image in enumerate(images):
        img_path = os.path.join(IMG_PATH, f"{os.path.basename(pdf_path)}_page_{i+1}.png")
        image.save(img_path, "PNG")
        image_paths.append(img_path)
        print(f"Seite {i+1} gespeichert als {img_path}")
    return image_paths

In [4]:
# Funktion zur Verarbeitung von PDFs
def process_pdfs(pdf_dir):
    print(f"Starte Verarbeitung der PDFs in {pdf_dir}")
    
    for filename in os.listdir(pdf_dir):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_dir, filename)
            print(f"Lese Datei: {pdf_path}")
            extract_images_from_pdf(pdf_path)

In [5]:
# Funktion zur Verarbeitung der Bilder
def process_images():
    extracted_data = []
    for img_file in os.listdir(IMG_PATH):
        if img_file.endswith(".png"):
            img_path = os.path.join(IMG_PATH, img_file)
            print(f"Sende Bild {img_path} an Pixtral")
            formula_latex = query_pixtral_with_ssh(img_path)
            if formula_latex:
                print(f"Formel extrahiert: {formula_latex}")
                extracted_data.append({
                    "image": img_file,
                    "formula": formula_latex
                })
    print(f"Speichere extrahierte Daten in {OUTPUT_JSON}")
    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(extracted_data, f, indent=4, ensure_ascii=False)


In [10]:
# Hauptausführung in Jupyter Notebook
process_pdfs(PDF_DIR)


Starte Verarbeitung der PDFs in c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\Paper
Lese Datei: c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\Paper\2103.11879v2.pdf
Konvertiere PDF c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\Paper\2103.11879v2.pdf in Bilder...
Seite 1 gespeichert als c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_1.png
Seite 2 gespeichert als c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_2.png
Seite 3 gespeichert als c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_3.png
Seite 4 gespeichert als c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_4.png
Seite 5 gespeichert als c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_5.png
Seite 6 gespeichert als c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_6.png
Seite 7 gespeichert als c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2

In [12]:
process_images()

Sende Bild c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_1.png an Pixtral
Verbindung zu Pixtral API unter http://127.0.0.1:8092/v1 mit API Key EMPTY***
SSH-Tunnel gestartet: localhost:8092 -> 8092
Verwendetes Modell: mistralai/Pixtral-12B-2409
Bild c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_1.png erfolgreich geladen und kodiert.
Antwort von Pixtral erhalten.
Formel extrahiert: Keine Formeln gefunden
Sende Bild c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_2.png an Pixtral
Verbindung zu Pixtral API unter http://127.0.0.1:8092/v1 mit API Key EMPTY***
SSH-Tunnel gestartet: localhost:8092 -> 8092
Verwendetes Modell: mistralai/Pixtral-12B-2409
Bild c:\Users\Felix Oppel\Odyss.AI\Odyss.AI.Backend.LLM\IMG\2103.11879v2.pdf_page_2.png erfolgreich geladen und kodiert.
Antwort von Pixtral erhalten.
Formel extrahiert: ```latex
\text{keine Formeln gefunden}
```
Sende Bild c:\Users\Felix Oppel\Odyss.AI\Ody

In [13]:
import pandas as pd
import json

# Lade die JSON-Datei
with open('extracted_formulas.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Wandelt die JSON-Daten in ein Pandas DataFrame um
df = pd.DataFrame(json_data)

# Ändert die Spaltennamen
df.columns = ['image', 'formel']

# Speichert das DataFrame als Excel-Datei
df.to_excel('output.xlsx', index=False)

print("Die Daten wurden erfolgreich in die Excel-Tabelle übertragen.")

Die Daten wurden erfolgreich in die Excel-Tabelle übertragen.
